In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

import scipy

import qgrid
import pandas as pd

In [3]:
#data taken from https://en.wikipedia.org/wiki/Van_der_Waals_constants_(data_page) using https://wikitable2csv.ggor.de/
#format: Element, a (L2bar/mol2), b (L/mol)

raw_data = [
"Acetic acid",17.71,0.1065,
"Acetic anhydride",20.158,0.1263,
"Acetone",16.02,0.1124,
"Acetonitrile",17.81,0.1168,
"Acetylene",4.516,0.0522,
"Ammonia",4.225,0.0371,
"Argon",1.355,0.03201,
"Benzene",18.24,0.1154,
"Bromobenzene",28.94,0.1539,
"Butane",14.66,0.1226,
"Carbon dioxide",3.640,0.04267,
"Carbon disulfide",11.77,0.07685,
"Carbon monoxide",1.505,0.03985,
"Carbon tetrachloride",19.7483,0.1281,
"Chlorine",6.579,0.05622,
"Chlorobenzene",25.77,0.1453,
"Chloroethane",11.05,0.08651,
"Chloromethane",7.570,0.06483,
"Cyanogen",7.769,0.06901,
"Cyclohexane",23.11,0.1424,
"Diethyl ether",17.61,0.1344,
"Diethyl sulfide",19.00,0.1214,
"Dimethyl ether",8.180,0.07246,
"Dimethyl sulfide",13.04,0.09213,
"Ethane",5.562,0.0638,
"Ethanethiol",11.39,0.08098,
"Ethanol",12.18,0.08407,
"Ethyl acetate",20.72,0.1412,
"Ethylamine",10.74,0.08409,
"Fluorobenzene",20.19,0.1286,
"Fluoromethane",4.692,0.05264,
"Freon",10.78,0.0998,
"Germanium tetrachloride",22.90,0.1485,
"Helium",0.0346,0.0238,
"Hexane",24.71,0.1735,
"Hydrogen",0.2476,0.02661,
"Hydrogen bromide",4.510,0.04431,
"Hydrogen chloride",3.716,0.04081,
"Hydrogen selenide",5.338,0.04637,
"Hydrogen sulfide",4.490,0.04287,
"Iodobenzene",33.52,0.1656,
"Krypton",2.349,0.03978,
"Mercury",8.200,0.01696,
"Methane",2.283,0.04278,
"Methanol",9.649,0.06702,
"Neon",0.2135,0.01709,
"Nitric oxide",1.358,0.02789,
"Nitrogen",1.370,0.0387,
"Nitrogen dioxide",5.354,0.04424,
"Nitrous oxide",3.832,0.04415,
"Oxygen",1.382,0.03186,
"Pentane",19.26,0.146,
"Phosphine",4.692,0.05156,
"Propane",8.779,0.08445,
"Radon",6.601,0.06239,
"Silane",4.377,0.05786,
"Silicon tetrafluoride",4.251,0.05571,
"Sulfur dioxide",6.803,0.05636,
"Tin tetrachloride",27.27,0.1642,
"Toluene",24.38,0.1463,
"Water",5.536,0.03049,
"Xenon",4.250,0.05105
]

In [4]:
data_array = np.array(raw_data)
data_reshaped = np.reshape(data_array, (-1,3)); #reshape in three columns

In [19]:
database = pd.DataFrame(data=data_reshaped, columns=["Element", "a (L2bar/mol2)", "b (L/mol)"])

#numpy converts all elements to 'object' and pandas interprets them as string, but I want a and b values to be float
database["a (L2bar/mol2)"] = pd.to_numeric(database["a (L2bar/mol2)"])
database["b (L/mol)"] = pd.to_numeric(database["b (L/mol)"])

In [20]:
grid_options = {
    # SlickGrid options
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': True,
    'defaultColumnWidth': 150,
    'rowHeight': 28,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'editable': True,
    'autoEdit': False,
    'explicitInitialization': True,

    # Qgrid options
    'maxVisibleRows': 5, #we have changed it to 5 (default = 15)
    'minVisibleRows': 5, #we have changed it to 5 (default = 8)
    'sortable': True,
    'filterable': True,
    'highlightSelectedCell': False,
    'highlightSelectedRow': True
}

qgrid_table = qgrid.show_grid(database, grid_options=grid_options)

In [21]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b k_B}`
        
   #Args:
   #    a: Term related with the attraction between particles in
   #    J m^3/ mol^2.\n
   #    b: Term related with the volume that is occupied by one 
   #    mole of the molecules in 10^-3 m^3/mol.\n
   #    
   #Returns:
   #    p_c: Critical pressure in Pa.\n
   #    v_c: Critical volume in m^3/mol.\n
   #    T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/k_B/N_A
    
    return p_c, v_c, T_c

In [22]:
for row in database.iterrows():
    data = row[1]
    element_name = data[0]
    a = data[1]
    b = data[2]    
    break

In [23]:
def generate_critic_points(df):
    #takes a dataframe containing three columns (element, a, b) adn returns four arrays: pc, vc, Tc and names
    
    pc = []
    vc = []
    Tc = []
    
    names = []
    
    for row in database.iterrows():
        data = row[1]
        
        names.append(data[0])
        
        a = float(data[1])
        b = float(data[2])       
        p, v, T = calculate_critic(a, b)
        
        pc.append(p)
        vc.append(v)
        Tc.append(T)
        
    return pc, vc, Tc, names

In [25]:
########################################
###########TOP BLOCK####################
########################################

top_block = widgets.HBox([], layout=widgets.Layout(width='50%', align_self='center'))

top_block.children = [qgrid_table]



########################################
###########MIDDLE BLOCK#################
########################################

middle_block = widgets.HBox([], layout=widgets.Layout(width='100%', align_self='center', align_content='center'))

fig_113_002 = bq.Figure(title='pc vs vc',
                marks=[],
                axes=[],
                animation_duration=500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=80, right=30),
                toolbar = True
    )
 
fig_113_003 = bq.Figure(title='Tc vs vc',
                marks=[],
                axes=[],
                animation_duration=500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=80, right=30),
                toolbar = True
    )

pc, vc, Tc, names = generate_critic_points(database)

scale_x_002 = bqs.LinearScale(min = min(vc), max = max(vc))
scale_x_003 = bqs.LinearScale(min = min(vc), max = max(vc))
scale_y_p = bqs.LinearScale(min = min(pc), max = max(pc))
scale_y_T = bqs.LinearScale(min = min(Tc), max = max(Tc))

axis_x_002 = bqa.Axis(scale=scale_x_002,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                ticks=[0,10,100],
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_x_003 = bqa.Axis(scale=scale_x_003,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                ticks=[0,10,100],
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')


axis_y_p = bqa.Axis(scale=scale_y_p,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

axis_y_T = bqa.Axis(scale=scale_y_T,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='T',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_113_002.axes = [axis_x_002, axis_y_p]
fig_113_003.axes = [axis_x_003, axis_y_T]

tt = widgets.Label("")

def hover_handler(self, content):
    tt.value = str(content.get('data').get('name'))

tb1 = Toolbar(figure=fig_113_002)
    
scatter_113_002 = bqm.Scatter(
    name = '',
    x = vc,
    y = pc,
    scales = {'x': scale_x_002, 'y': scale_y_p}, 
    default_opacities = [0.2],
    visible = True,
    colors = [],
    names = names,
    display_names = False,
    labels=[],
    tooltip = tt
) 

scatter_113_002.on_hover(hover_handler)

scatter_113_003 = bqm.Scatter(
    name = '',
    x = vc,
    y = Tc,
    scales = {'x': scale_x_003, 'y': scale_y_T}, 
    default_opacities = [0.2],
    visible = True,
    colors = [],
    names = names,
    display_names = False,
    labels=[],
    tooltip = tt
) 

scatter_113_003.on_hover(hover_handler)

fig_113_002.marks = [scatter_113_002]
fig_113_003.marks = [scatter_113_003]

middle_block.children = [fig_113_002,
                         #widgets.VBox([tb1], layout=widgets.Layout(display='flex')),
                         fig_113_003]
########################################
###########MAIN BLOCK###################
########################################

widgets.VBox([top_block,
             middle_block,],
            layout=widgets.Layout(align_content='center'))

In [14]:
database.dtypes

Element           object
a (L2bar/mol2)    object
b (L/mol)         object
dtype: object